# 머신러닝 기반 데이터분석_윤혜인

**Loading Libraries 라이브러리 로딩**

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import warnings
warnings.simplefilter(action='ignore')
import matplotlib.pyplot as plt
from tqdm import tqdm
from datetime import datetime
import json
from sklearn.preprocessing import LabelEncoder

**Get Datasets 데이터 가져오기**

In [ ]:
train = pd.read_csv('../input/tmdb-box-office-prediction/train.csv')
train.info()

In [ ]:
test = pd.read_csv('../input/tmdb-box-office-prediction/test.csv')
test.info()

**Column Explanation 컬럼 설명**

- DataDescription id 각 영화의 유니크한 숫자
- belongs_to_collection TMDB id, 이름, 영화포스터, URL 정보(JSON Format)
- budget 영화 예산 (달러), 0은 알려지지 않은 것을 뜻함
- genres 장르 이름과 TMDB id(JSON Format)
- homepage 공식 홈페이지 주소
- imdb_id 영화 커뮤니티인 imdb의 id (string)
- original_language 원어를 나타내는 두 자리 코드 (예 : en = English)
- original_title 영화의 원 제목
- overview 영화의 간단한 설명
- popularity 실수(float)로 표현된 영화의 평판/인기
- poster_path 영화 포스터의 URL 경로
- production_companies 제작사 이름과 TMDB id(JSON Format)
- production_countries 제작 국가의 두 자리 코드(JSON Format)
- release_date 영화 개봉일 mm/dd/yy
- runtime 영화 러닝타임 (분/정수)
- spoken_languages 언어를 표현하는 두 자리 코드
- status 영화 개봉 여부
- tagline 영화의 태그라인(슬로건/부제?)
- title 영어 제목
- keywords 모든 키워드와 TMDB id
- cast 모든 등장인물의 이름, 캐릭터 이름, 성별(female = 1, male = 2), TMDB id(JSON Format)
- crew 이름, TMDB id, 모든 스탭들의 프로필 경로
- revenue 총 이익 (달러)

In [ ]:
train.head()

In [ ]:
test.head()

In [ ]:
train.describe(include='all')

In [ ]:
test.describe(include='all')

**Count of missing values in each column in train and test**

자료의 각 컬럼에서 결측값을 센다

In [ ]:
train.isna().sum()

In [ ]:
test.isna().sum()

**plot Budget vs Revenue**

예산과 이익을 그래프로(seaborn) 나타내기

In [ ]:
sns.jointplot(x='budget', y='revenue', data=train, height=11, ratio=4, color='g')
plt.show()

**plot Popularity vs Revenue**

평판과 이익을 그래프로(seaborn) 나타내기

In [ ]:
sns.jointplot(x='popularity', y='revenue', data=train, height=11, ratio=4, color='g')
plt.show()

**plot Runtime vs Revenue**

상영시간과 이익을 그래프(seaborn)로 나타내기

In [ ]:
sns.jointplot(x='runtime', y='revenue', data=train, height=11, ratio=4, color='g')
plt.show()

**Revenue (Target Variable) Distribution**

이익 분포

In [ ]:
sns.distplot(train.revenue)

❗ 분포가 치우쳐 있음을 볼 수 있다

In [ ]:
train.revenue.describe()

**Calulate log of Revenue**

분포를 로그로 계산하여 표준편차 그리기

In [ ]:
train['logRevenue'] = np.log1p(train['revenue'])
sns.distplot(train['logRevenue'])

❗ 분포가 정규분포에 더 가깝게 변환됨

**Get Month, Day, Year, Week day, Quarter from release date**

개봉일의 세부 데이터 만들기

In [ ]:
# 년도 자료는 2자리 수로 제공되지만, 몇몇 레코드에서 4자리로 표현하므로 둘을 나눠서 해야 함

train[['release_month','release_day','release_year']] = train['release_date'].str.split('/', expand=True).replace(np.nan, -1).astype(int)

# 년도 자료가 4 자리인 것 때문에 추가 된 라인 
train.loc[(train['release_year'] <= 19) & (train['release_year'] <100), 'release_year' ] += 2000
train.loc[(train['release_year'] > 19) & (train['release_year'] <100), 'release_year' ] += 1900

releaseDate = pd.to_datetime(train['release_date'])
train['release_dayofweek'] = releaseDate.dt.dayofweek
train['release_quarter'] = releaseDate.dt.quarter

**plot Release year count**

개봉 년도 그래프

In [ ]:
plt.figure(figsize = (20,12))
sns.countplot(train['release_year'].sort_values())
plt.title('Movie Release count by Year', fontsize = 20)
loc, labels = plt.xticks()
plt.xticks(fontsize = 12, rotation = 90)
plt.show()

**plot Release month count**

개봉 월 그래프

In [ ]:
plt.figure(figsize=(20,12))
sns.countplot(train['release_month'].sort_values())
plt.title('Release Month Count', fontsize = 20)
loc, labels = plt.xticks()
loc, labels = loc, ["Jan", "Feb", "Mar", "Apr", "May", "Jun", "Jul", "Aug", "Sep", "Oct", "Nov", "Dec"]
plt.xticks(loc, labels, fontsize=20)
plt.show()

**plot Release day count**

일자별 그래프

In [ ]:
plt.figure(figsize=(20,12))
sns.countplot(train['release_day'].sort_values())
plt.title('Release Day count',fontsize=20)
plt.xticks(fontsize=20)
plt.show()

**plot Release day of week**

요일별 그래프

In [ ]:
plt.figure(figsize=(20,12))
sns.countplot(train['release_dayofweek'].sort_values())
plt.title('Release Dayof count',fontsize=20)
loc, labels = plt.xticks()
loc, labels = loc, ["Mon","Tue","Wed","Thu","Fri","Sat","Sun"]
plt.xticks(loc, labels, fontsize=20)
plt.show()

**plot Release quater count**

분기별 그래프

In [ ]:
plt.figure(figsize=(20,12))
sns.countplot(train['release_quarter'].sort_values())
plt.title('Release Quater count',fontsize=20)
plt.xticks(fontsize=20)
plt.show()

**Release Year by Revenue**

이익순으로 보는 년도 그래프

In [ ]:
train['meanRevenueByYear'] = train.groupby('release_year')['revenue'].aggregate('mean')
train['meanRevenueByYear'].plot(figsize=(15,10),color='g')
plt.xticks(np.arange(1920,2018,4)) # 1920부터 2018까지 4 단위로 x축 나오도록 함
plt.xlabel('ReleaseYear')
plt.ylabel('Revenue')
plt.title('Movie Mean Revenue By Year', fontsize=20)
plt.show()

**Release Month by Revenue Plot**

이익순으로 보는 월별 그래프

In [ ]:
train['meanRevenueByMonth'] = train.groupby('release_month')['revenue'].aggregate('mean')
train['meanRevenueByMonth'].plot(figsize=(15,10),color='g')
plt.xlabel('ReleaseMonth')
plt.ylabel('Revenue')
plt.title('Movie Mean Revenue By Month', fontsize=20)
plt.show()